Загрузка библиотек

In [5]:
import os
import sys
import pandas as pd
import numpy as np
import datetime
from matplotlib import pyplot as plt
import seaborn as sns
from IPython.display import display, Markdown


from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import f1_score
from sklearn import model_selection, linear_model, metrics
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

plt.rcParams.update({'figure.max_open_warning': 0})
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

from sklearn.metrics import classification_report, confusion_matrix, f1_score, precision_score, recall_score

from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression,Lasso, Ridge, SGDRegressor
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM, SVR
from sklearn.neighbors import KNeighborsRegressor

import pickle

Загрузка данных

In [6]:
path = r"C:/Users/user/Desktop/test.pkl/test.pkl"
with open(path, "rb") as f:
    list_of_df = pickle.load(f)

In [7]:
len(list_of_df)

34

In [8]:
pd.concat(list_of_df)

,Accelerometer1RMS,Accelerometer2RMS,Current,Pressure,Temperature,Thermocouple,Voltage,Volume Flow RateRMS
datetime,,,,,,,,
2020-03-09 12:14:36,0.027429,0.040353,0.770310,0.382638,71.2129,25.0827,219.789,32.0000
2020-03-09 12:14:37,0.027269,0.040226,1.096960,0.710565,71.4284,25.0863,233.117,32.0104
2020-03-09 12:14:38,0.027040,0.039773,1.140150,0.054711,71.3468,25.0874,234.745,32.0000
2020-03-09 12:14:39,0.027563,0.040313,1.108680,-0.273216,71.3258,25.0897,205.254,32.0104
2020-03-09 12:14:41,0.026570,0.039566,0.704404,0.382638,71.2725,25.0831,212.095,33.0000
...,...,...,...,...,...,...,...,...
2020-03-09 11:34:30,0.027081,0.038897,0.624425,0.054711,71.3921,25.4292,232.531,32.0000
2020-03-09 11:34:31,0.026692,0.039526,0.663211,0.382638,71.3944,25.4293,234.818,32.0057
2020-03-09 11:34:32,0.026836,0.039188,0.771298,0.054711,71.4257,25.4274,230.235,32.9891


Подготовка данных

Лучше сделать фильтрацию без удаления нереальных значений, иначе предсказание будет отличаться по размерам от целевой переменной в лидерборде


In [9]:
for n in range(len(list_of_df)):
    df = list_of_df[n]
    for col in df.columns:
        df[df < 0] = 0
    list_of_df[n] = df    

In [10]:
pd.concat(list_of_df)

,Accelerometer1RMS,Accelerometer2RMS,Current,Pressure,Temperature,Thermocouple,Voltage,Volume Flow RateRMS
datetime,,,,,,,,
2020-03-09 12:14:36,0.027429,0.040353,0.770310,0.382638,71.2129,25.0827,219.789,32.0000
2020-03-09 12:14:37,0.027269,0.040226,1.096960,0.710565,71.4284,25.0863,233.117,32.0104
2020-03-09 12:14:38,0.027040,0.039773,1.140150,0.054711,71.3468,25.0874,234.745,32.0000
2020-03-09 12:14:39,0.027563,0.040313,1.108680,0.000000,71.3258,25.0897,205.254,32.0104
2020-03-09 12:14:41,0.026570,0.039566,0.704404,0.382638,71.2725,25.0831,212.095,33.0000
...,...,...,...,...,...,...,...,...
2020-03-09 11:34:30,0.027081,0.038897,0.624425,0.054711,71.3921,25.4292,232.531,32.0000
2020-03-09 11:34:31,0.026692,0.039526,0.663211,0.382638,71.3944,25.4293,234.818,32.0057
2020-03-09 11:34:32,0.026836,0.039188,0.771298,0.054711,71.4257,25.4274,230.235,32.9891


Генерация новых признаков

In [11]:
for ind in range(len(list_of_df)):
    # разница температур (если признаки скоррелированы и с одним из них что-то происходит, то покажет наличие аномалии)
    list_of_df[ind]["Temperature_diff"] = list_of_df[ind]['Temperature'] - list_of_df[ind]['Thermocouple'] 
    # разница акселлерометров (если признаки скоррелированы и с одним из них что-то происходит, то покажет наличие аномалии)
    list_of_df[ind]["Accel_diff"] = list_of_df[ind]['Accelerometer1RMS'] - list_of_df[ind]['Accelerometer2RMS'] 
    # сглаживания
    list_of_df[ind]["Volume Flow RateRMS_10mean"] = list_of_df[ind]["Volume Flow RateRMS"].rolling(window = 10, min_periods=0).mean()
    

In [12]:
pd.concat(list_of_df)

,Accelerometer1RMS,Accelerometer2RMS,Current,Pressure,Temperature,Thermocouple,Voltage,Volume Flow RateRMS,Temperature_diff,Accel_diff,Volume Flow RateRMS_10mean
datetime,,,,,,,,,,,
2020-03-09 12:14:36,0.027429,0.040353,0.770310,0.382638,71.2129,25.0827,219.789,32.0000,46.1302,-0.012924,32.000000
2020-03-09 12:14:37,0.027269,0.040226,1.096960,0.710565,71.4284,25.0863,233.117,32.0104,46.3421,-0.012956,32.005200
2020-03-09 12:14:38,0.027040,0.039773,1.140150,0.054711,71.3468,25.0874,234.745,32.0000,46.2594,-0.012733,32.003467
2020-03-09 12:14:39,0.027563,0.040313,1.108680,0.000000,71.3258,25.0897,205.254,32.0104,46.2361,-0.012749,32.005200
2020-03-09 12:14:41,0.026570,0.039566,0.704404,0.382638,71.2725,25.0831,212.095,33.0000,46.1894,-0.012996,32.204160
...,...,...,...,...,...,...,...,...,...,...,...
2020-03-09 11:34:30,0.027081,0.038897,0.624425,0.054711,71.3921,25.4292,232.531,32.0000,45.9629,-0.011815,32.200040
2020-03-09 11:34:31,0.026692,0.039526,0.663211,0.382638,71.3944,25.4293,234.818,32.0057,45.9651,-0.012834,32.200610
2020-03-09 11:34:32,0.026836,0.039188,0.771298,0.054711,71.4257,25.4274,230.235,32.9891,45.9983,-0.012352,32.299520


Обучение модели: случайный лес

In [ ]:
# параметры для моделирования и подбора порога
N_STEPS = 3
Q = 0.99

In [ ]:
def split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix, :]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)